In [26]:
%%writefile similarity.py
def app(text1, text2):
    import re, math
    from collections import Counter
    WORD = re.compile(r'\w+')
    def get_cosine(vec1, vec2):
         intersection = set(vec1.keys()) & set(vec2.keys())
         numerator = sum([vec1[x] * vec2[x] for x in intersection])

         sum1 = sum([vec1[x]**2 for x in vec1.keys()])
         sum2 = sum([vec2[x]**2 for x in vec2.keys()])
         denominator = math.sqrt(sum1) * math.sqrt(sum2)

         if not denominator:
            return 0.0
         else:
            return float(numerator) / denominator
    def text_to_vector(text):
         words = WORD.findall(text)
         return Counter(words)
    def run(txt1, txt2):
        vector1 = text_to_vector(txt1)
        vector2 = text_to_vector(txt2)
        cosine = get_cosine(vector1, vector2)
        return cosine
    return run(text1, text2)

Overwriting similarity.py


In [27]:
%%writefile my_flask_app.py 
from flask import Flask, Response, jsonify 
from flask_restplus import Api, Resource, fields, reqparse 
from flask_cors import CORS, cross_origin import os 
# the app 
app = Flask(__name__) 
CORS(app) 
api = Api(app, version='1.0', title='APIs for finding text similarity using Python ', validate=False) ns = api.namespace('similarity', 'Returns a similarity index of two sentences') 
# load the algo 
from similarity import app as algo 
''' We import our function `app` from the file similarity.py. You create all the classes and functions that you want in that file, and import them into the app. ''' 
# model the input data 
model_input = api.model('Sentences', { "text1": fields.String, "text2": fields.String})
# the input data type here is Integer. You can change this to whatever works for your app. 
# On Bluemix, get the port number from the environment variable PORT # When running this app on the local machine, default to 8080 
port = int(os.getenv('PORT', 8080)) 
# The ENDPOINT 
@ns.route('/similarity') 
# the endpoint 
class SIMILARITY(Resource): 
    @api.response(200, "Success", model_input)   
    @api.expect(model_input)
    def post(self):
        parser = reqparse.RequestParser()
        parser.add_argument('text1', type=str)
        parser.add_argument('text2', type=str)
        args = parser.parse_args()
        inp1 = str(args["text1"])
        inp2 = str(args["text2"])
        result = algo(inp1, inp2) 
        return jsonify({"similarity": result}) 
if __name__ == '__main__': 
    app.run(host='0.0.0.0', port=port, debug=False) # deploy with debug=False

Overwriting my_flask_app.py


In [28]:
%%writefile manifest.yml
 --- applications:
 - name: SIMILARITY_FINDER
 - random-route: true
 - memory: 256M

Overwriting manifest.yml


In [29]:
%%writefile Procfile 
web: python my_flask_app.py

Overwriting Procfile


In [30]:
%%writefile README.md 
"Getting Started with Python Algos on Bluemix"

Overwriting README.md


In [31]:
%%writefile requirements.txt 
Flask==0.11.1 
cloudant==2.4.0 
flasgger==0.6.4 
Flask-Cors==3.0.2 
Flask-RESTful==0.3.6 
flask-restplus==0.9.2 
gevent==1.2.1

Overwriting requirements.txt


In [32]:
%%writefile setup.py 
""" Hello World app for deploying Python functions as APIs on Bluemix """ 
# Always prefer setuptools over distutils 
from setuptools import setup, find_packages 
from codecs import open 
from os import path 
here = path.abspath(path.dirname(__file__)) 
with open(path.join(here, 'README.md'), encoding='utf-8') as f:
    long_description = f.read() 
setup( 
name='similarity', 
version='1.0.0', 
description='Running Python apps on Bluemix', long_description=long_description, license='Apache-2.0' 
)

Overwriting setup.py


In [33]:
%%writefile LICENSE 
"""Apache License Version 2.0, January 2004 http://www.apache.org/licenses/ TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION ..."""

Overwriting LICENSE
